In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
from pathlib import Path

In [ ]:
from smtag.config import config
config

In [ ]:
from transformers import __version__
__version__

In [ ]:
config.tokenizer.mask_token

## Extracting examples for LM

In [ ]:
from smtag.extract import ExtractorXML

In [ ]:
! dir /data/xml/emboj_all

In [ ]:
! rm -fr /data/text/emboj_abstract_test

In [ ]:
corpus = "/data/xml/emboj_all"
text_examples = "/data/text/emboj_abstract_test"
xpath = ".//abstract"
sentence_level = False
keep_xml = False
inclusion_probability = 1.0

In [ ]:
extractor_lm = ExtractorXML(
    corpus,
    destination_dir=text_examples,
    sentence_level=sentence_level,
    xpath=xpath,
    keep_xml=keep_xml,
    inclusion_probability=inclusion_probability
)

In [ ]:
extractor_lm.extract_from_corpus()

same via CLI:

```bash
python -m smtag.cli.prepro.extract /data/xml/mini/ /data/text/mini --xpath ".//abstract"
```

## Preparing tokenized dataset for LM

In [ ]:
from smtag.dataprep import PreparatorLM

In [ ]:
tokenized_examples = "/data/json/emboj_abstract_test"

In [ ]:
! rm -fr /data/json/emboj_abstract_test

In [ ]:
prep_lm = PreparatorLM(
    text_examples,
    tokenized_examples,
    max_length=config.max_length
)

In [ ]:
prep_lm.run()

same vie CLI:
    
```bash
python -m smtag.cli.lm.dataprep /data/text/mini /data/json/mini
```

## Train LM

In [ ]:
from smtag.train.train_lm import (
    train as train_lm,
    TrainingArgumentsLM
)

In [ ]:
no_cache = False
loader_path = "./smtag/loader/loader_lm.py"  # includes a loading script with the same name ./lm/lm.py
data_config_name = "DET"
tokenizer = config.tokenizer  # tokenizer has to be the same application-wide
model_type = "Autoencoder"
from_pretrained = config.from_pretrained

In [ ]:
tokenizer.mask_token = '<mask>'
# tokenizer.unk_token = '<unk>'

In [ ]:
training_args_tokcl = TrainingArgumentsLM(
    num_train_epochs = 30,
    logging_steps = 10,
    per_device_train_batch_size=4, #32,
    per_device_eval_batch_size=3, #32,
)
training_args_tokcl

In [ ]:
train_lm(
    training_args_tokcl,
    loader_path,
    data_config_name,
    tokenized_examples,
    no_cache,
    tokenizer,
    model_type,
    from_pretrained
)

train with command line:

    python -m smtag.cli.lm.train "./smtag/loader/loader_lm.py" DET --data_dir /data/json/emboj_abstract_test --no_cache --per_device_train_batch_size=4 --per_device_eval_batch_size=4